In [4]:
import numpy as np
import copy
import random 
from pathlib import Path

In [6]:
class KPBB:
    def __init__(self, cap, values, weights): # Initialisation des données
        self.capacity = cap
        self.values = values
        self.weights = weights
        self.objects = [((v, w), i) for i, (v, w) in enumerate(zip(values, weights))]
        self.best_solution=0
        self.best_value=0
       

    def initial_solution(self):
        solution = []
        items = self.objects.copy()
        c = self.capacity
        while c > 0 and len(items) > 0:
            myobject = items.pop(np.random.randint(0, len(items)-1))
            if myobject[0][1] <= c:
                solution.append(myobject[1])
            c -= myobject[0][1]
        return solution

    def voisinage(self,solution):

        objects_no_solution = [i for i in range(len(self.objects)) if i not in solution]
        o = objects_no_solution.pop(random.randint(0, len(objects_no_solution)-1))
        solution.append(o)
        if len(solution) == 1:
            return solution
        while self.evalution_weight(solution) > self.capacity and len(solution) > 1:
            solution.pop(random.randint(0, len(solution)-2))
        return solution
    def Metro_criterion(self, delta, t):
          
          return random.random() < np.exp(delta/t)

    def evaluation_values(self, solution):
            val = 0
            for obj in solution:
                val += self.objects[obj][0][0]
            return val

    def evalution_weight(self, solution):

        weight = 0
        for obj in solution:
            weight += self.objects[obj][0][1]
        return weight
    
    def temperature(self,t,k,step=10):
        return t - (k%step == 0) * step

    def solve(self, T0=300):
        S0 = self.initial_solution()
        t = T0
        i = 0
        last_best = 0
        max_iter = 1000
        self.best_solution  = S0
        self.best_value = self.evaluation_values(self.best_solution)
        while i < max_iter and t > 0:
            S1 = self.voisinage(S0)
            delta = self.evaluation_values(S1) - self.evaluation_values(S0)
            if delta > 0:
                S0 = S1
            elif self.Metro_criterion(delta, t):
                S0 = S1
            if self.evaluation_values(S0) > self.best_value:
                self.best_solution = S0.copy()
                self.best_value = self.evaluation_values(self.best_solution)
            
            t = self.temperature(t, i)
            i += 1

capacite = 10
values = [40, 50, 100, 95, 30]
weights = [2, 3, 1, 5, 3]
kpp = KPBB(capacite, values, weights)
kpp.solve()


In [7]:
kpp.best_solution

[1, 3, 2]

In [8]:
kpp.best_value

245

In [9]:
def execute_all(directory):
    p=Path('./instances_01_KP/'+directory)
    print('Fichier        Solution     Optimum')
    nb=0
    for f in p.iterdir():
        
        first_row=np.loadtxt(f.absolute(),max_rows=1)
        fich=np.loadtxt(f.absolute(),max_rows=int(first_row[0])+1)
        clf=KPBB(fich[0][1],fich[1:,0],fich[1:,1])
        clf.solve()
        print(f'{f.name}-> {clf.best_solution} -> {clf.best_value}')
        nb+=1
        if nb==7:
            return

In [10]:
execute_all('low-dimensional')

Fichier        Solution     Optimum
f2_l-d_kp_20_878-> [2, 12, 6, 19, 14, 4, 15, 3, 1, 16, 9, 18, 10, 17, 11, 0, 8] -> 995.0
f8_l-d_kp_23_10000-> [1, 6, 4, 5, 14, 8, 3, 0, 11, 16, 2] -> 9756.0
f4_l-d_kp_4_11-> [1, 3] -> 23.0
f10_l-d_kp_20_879-> [3, 1, 10, 8, 4, 5, 7, 16, 17, 15, 11, 13, 0, 12, 2, 6, 18] -> 998.0
f5_l-d_kp_15_375-> [14, 2, 11, 9, 10, 6, 13, 4] -> 450.669881
f3_l-d_kp_4_20-> [1, 0, 3] -> 35.0
f1_l-d_kp_10_269-> [7, 9, 8, 2, 1, 3] -> 295.0


In [11]:
execute_all('large_scale')

Fichier        Solution     Optimum
knapPI_3_100_1000_1-> [23, 43, 6, 74] -> 1381.0
knapPI_3_2000_1000_1-> [1751, 1600, 1645, 928, 1628, 1302, 489, 1521, 1863, 474, 614, 873, 1753, 868, 417, 1729, 606, 1862, 1630, 797, 1458, 933, 1737] -> 12045.0
knapPI_3_500_1000_1-> [183, 304, 437, 43, 271, 444, 74, 56] -> 3313.0
knapPI_3_200_1000_1-> [120, 109, 171, 135] -> 1383.0
knapPI_2_5000_1000_1-> [4130, 3267, 3233, 586, 2714, 2542, 3342, 2374, 77, 2918, 3081, 987, 4416, 2516, 3671, 3146, 1369, 1041, 41, 2493, 549, 3469, 2194, 770, 2819, 3997, 2466, 1066, 3072, 2857, 273, 1110, 2050, 1797, 4032, 4218, 2267, 2324, 4371, 3574, 2673, 4672, 1310, 2664, 4209, 3281, 3967, 3025, 1421, 3871, 1780, 3730, 585, 3448, 3407, 3165, 3167] -> 25417.0
knapPI_1_500_1000_1-> [221, 245, 125, 117, 281, 177, 168, 271] -> 4732.0
knapPI_3_10000_1000_1-> [985, 3966, 1745, 3715, 435, 5652, 4846, 2833, 9684, 1448, 2163, 5781, 3356, 5706, 2075, 1578, 5774, 2838, 9022, 4287, 4128, 6862, 7551, 1786, 4725, 2791, 5757, 2043,